Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

### Table of Contents
1. [Introduction]()
2. [Import all Dependencies](1.0.1)
3. [Setup Workspace and Experiment](1.1)
4. [Dataset](1.2)
5. [Hyperdrive Configuration](1.3)
6. [Model Deployment](1.6)
7. [Test](1.7)
8. [Get Logs](1.8)
9. [Delete Resources](1.9)

## Introduction

This is a Capstone project in fulfilment of the Udacity Azure ML Nanodegree. 


This project is aimed at demonstrating the capabilities of the Azure ML studio in training a model and deploying it. There are two ways Azure ML studio achieves this: one is through AUTOML, a codeless configuration that automates machine learning. Another, is the HYPERDRIVE, a custom hyperparameter tuning functionality for optimizing a ML model's performance. Then, from any of these two functionalities of the Azure ML studio, a production model will emerge to enable us explore the Azure ML End-to-End production pipeline solution for enabling interaction between a deployed model and other web services.


In this demo, the HYPERDRIVE tuned XGBOOST model with hyperparameters of *n_estimators = 1000*, *max-depth = 5*, and *Subsample = 0.8* emerged as the best performing model with an accuracy of 99.988%. 

### Import Dependencies

In [1]:
# System libraries
import os
import csv
import shutil
import logging
# Conda dependency libraries
import numpy as np
import pandas as pd
import pkg_resources
from matplotlib import pyplot as plt
# importing core Azure libraries
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
# Compute target libraries
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Hyperdrive run libraries
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.widgets import RunDetails
# Environment variables libraries
from azureml.core.environment import Environment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
# Library for Saving files
import joblib
# Webservice libraries
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
#Libraries for testing deployed webservice
import json
import requests
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Setup Workspace and Experiment

### Define Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-137067
aml-quickstarts-137067
southcentralus
aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee


### Create Experiment

In [3]:
# choose a name for experiment
experiment_name = 'liver-disease-hyperparameter'

experiment=Experiment(ws, experiment_name)

### Create Compute Cluster

In [4]:
#create the cluster
#name the CPU cluster
cpu_cluster_name = "notebook137067"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


### Setup Environment Folders


In [5]:
#project folder
project_folder = './project_files'
os.makedirs(project_folder, exist_ok=True)

# Copy the train.py file to the project folder
shutil.copy('./train.py', project_folder)

#Model folder
output_folder = './outputs'
os.makedirs(output_folder, exist_ok=True)

#test folder
result_folder = './results'
os.makedirs(result_folder, exist_ok=True)


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

**Overview**


There is an increasing number of patients with liver disease in recent time due to life style and living habits such as excessive alcohol consumption, inhale of harmful gases, excessive weight gain, intake of contaminated food, abuse of drugs. This dataset is aimed at helping doctors during clinical diagnosis of liver disease to elevate burden and the stress involved in analyzing every single patients’ information. Therefore, the goal is to create a classifier that predicts whether a subject is healthy (non-liver patient) or ill (liver patient) based on some clinical and demographic features which are: age, gender, total Bilirubin, direct Bilirubin, total proteins, albumin, A/G ratio, SGPT, SGOT and Alkphos.

data is available at "https://raw.githubusercontent.com/chollette/Liver-Disease-Classification-Azure-ML-Capstone-Project/master/starter_file/data/Liver%20Patient%20Dataset%20(LPD)_train.csv"

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# SKLearn estimator for use with train.py
#TODO: Create your estimator and hyperdrive config
script_params = {
    '--n_estimators': 10,
    '--max_depth': 3,
    '--subsample': 0.2,
}

estimator = SKLearn(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train.py',
                    pip_packages = ["xgboost"], 
                   )
#run = experiment.submit(estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
#RunDetails(run).show()


In [8]:
#run.cancel()

### Define Termination Policy

In [9]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


### Set Parameter sampler

In [10]:

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    "--n_estimators": choice([10, 50, 100, 500, 1000, 5000]),
    "--max_depth": choice([3, 4, 5, 6, 7, 8, 9, 10]),
    "--subsample": choice([0.5,0.6,0.7,0.8,0.9,1])
    }
)


### Configure Hyperdrive and Run

In [11]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name="accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [12]:
# Hperdrive run 
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_config)


The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--n_estimators', '--max_depth', '--subsample'] is the list of overridden parameter(s).


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

### Overview
This HYPERDRIVE is am XGBOOST hyperparameter tunning experiment built by considering the best parameters observed from the AUTOML to xgboost log to result in the accuracy achieved. Given the main contributing parameters such as n-estimators, max_depth and subsample, which from literature are significant for optimizing the XGBOOST performance, we run different values of these hyperparameters to achieve better accruacy than the AUTOML. From our experiment, it shows that AUTOML can serve as a starting point for ML experts and novice to save time training different models.

In [13]:
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
hyperdrive_run.cancel()

In [15]:
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_81238136-2473-42f8-af5b-91b1a142dddd
Web View: https://ml.azure.com/experiments/liver-disease-hyperparameter/runs/HD_81238136-2473-42f8-af5b-91b1a142dddd?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136498/workspaces/quick-starts-ws-136498

Execution Summary
RunId: HD_81238136-2473-42f8-af5b-91b1a142dddd
Web View: https://ml.azure.com/experiments/liver-disease-hyperparameter/runs/HD_81238136-2473-42f8-af5b-91b1a142dddd?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-136498/workspaces/quick-starts-ws-136498



### Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [16]:
#retreive best run
best_hdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hdrive_run_metrics = hyperdrive_run.get_metrics()
best_hdrive_run_children = hyperdrive_run.get_children_sorted_by_primary_metric()

In [36]:
print(best_hdrive_run)
print(best_hdrive_run.get_details()['runDefinition']['arguments'])

Run(Experiment: liver-disease-hyperparameter,
Id: HD_81238136-2473-42f8-af5b-91b1a142dddd_7,
Type: azureml.scriptrun,
Status: Completed)
['--max_depth', '5', '--n_estimators', '1000', '--subsample', '0.8']


In [18]:
print(best_hdrive_run_metrics)

{'HD_81238136-2473-42f8-af5b-91b1a142dddd_19': {'Number of Trees:': 10, 'Max depth:': 10, 'Subsample of Dataset:': 0.6, 'accuracy': 0.9860088365243005}, 'HD_81238136-2473-42f8-af5b-91b1a142dddd_18': {'Number of Trees:': 500, 'Max depth:': 4, 'Subsample of Dataset:': 0.9, 'accuracy': 0.9996318114874816}, 'HD_81238136-2473-42f8-af5b-91b1a142dddd_17': {'Number of Trees:': 1000, 'Max depth:': 5, 'Subsample of Dataset:': 0.5, 'accuracy': 0.9998772704958272}, 'HD_81238136-2473-42f8-af5b-91b1a142dddd_16': {'Number of Trees:': 10, 'Max depth:': 8, 'Subsample of Dataset:': 0.8, 'accuracy': 0.9802405498281787}, 'HD_81238136-2473-42f8-af5b-91b1a142dddd_14': {'Number of Trees:': 100, 'Max depth:': 5, 'Subsample of Dataset:': 1.0, 'accuracy': 0.9970544918998527}, 'HD_81238136-2473-42f8-af5b-91b1a142dddd_13': {'Number of Trees:': 10, 'Max depth:': 3, 'Subsample of Dataset:': 0.8, 'accuracy': 0.7674275895925381}, 'HD_81238136-2473-42f8-af5b-91b1a142dddd_15': {'Number of Trees:': 50, 'Max depth:': 7, 

In [19]:
#Retireve metrics by run ID
for x, y in best_hdrive_run_metrics.items():
    print(x,"n_estimators:", y['Number of Trees:'], "   ", "Accuracy:",  y['accuracy'])
    print(x, "max_depth:", y['Max depth:'], "   ", "Accuracy:",  y['accuracy'])
    print(x, "subsample:", y['Subsample of Dataset:'], "   ", "Accuracy:",  y['accuracy'])

HD_81238136-2473-42f8-af5b-91b1a142dddd_19 n_estimators: 10     Accuracy: 0.9860088365243005
HD_81238136-2473-42f8-af5b-91b1a142dddd_19 max_depth: 10     Accuracy: 0.9860088365243005
HD_81238136-2473-42f8-af5b-91b1a142dddd_19 subsample: 0.6     Accuracy: 0.9860088365243005
HD_81238136-2473-42f8-af5b-91b1a142dddd_18 n_estimators: 500     Accuracy: 0.9996318114874816
HD_81238136-2473-42f8-af5b-91b1a142dddd_18 max_depth: 4     Accuracy: 0.9996318114874816
HD_81238136-2473-42f8-af5b-91b1a142dddd_18 subsample: 0.9     Accuracy: 0.9996318114874816
HD_81238136-2473-42f8-af5b-91b1a142dddd_17 n_estimators: 1000     Accuracy: 0.9998772704958272
HD_81238136-2473-42f8-af5b-91b1a142dddd_17 max_depth: 5     Accuracy: 0.9998772704958272
HD_81238136-2473-42f8-af5b-91b1a142dddd_17 subsample: 0.5     Accuracy: 0.9998772704958272
HD_81238136-2473-42f8-af5b-91b1a142dddd_16 n_estimators: 10     Accuracy: 0.9802405498281787
HD_81238136-2473-42f8-af5b-91b1a142dddd_16 max_depth: 8     Accuracy: 0.980240549828

In [20]:
print(best_hdrive_run.get_details()['runDefinition']['arguments'])

['--max_depth', '5', '--n_estimators', '1000', '--subsample', '0.8']


In [21]:
print(best_hdrive_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_c82f4b2bd092de200b11b960ac65d4a900365f77b6a10a403c8a59bef36d3b15_d.txt', 'azureml-logs/65_job_prep-tvmps_c82f4b2bd092de200b11b960ac65d4a900365f77b6a10a403c8a59bef36d3b15_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_c82f4b2bd092de200b11b960ac65d4a900365f77b6a10a403c8a59bef36d3b15_d.txt', 'logs/azureml/106_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [22]:
#TODO: Retrieve the best model
best_hdrive_run.download_file("outputs/model.joblib", )

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### Register the Fitted Model for Deployment

In [23]:
#Register model
model = best_hdrive_run.register_model(model_name='model',
                           model_path='outputs/model.joblib',
                           tags=best_hdrive_run.get_metrics())
print(model.name, model.id, model.version, sep='\t')

model	model:1	1


### Compute Environment and Variables for Deployment
We also need to create an environment file so that Azure Machine Learning can install the necessary packages in the Docker image which are required by your scoring script.

In [6]:
#set run configuration
run_config = RunConfiguration()

# Create the environment
myenv = Environment(name="myenv")

#You can manage an environment by registering it.
myenv.register(workspace=ws)
# Enable Docker and reference an image
myenv.docker.enabled = True


# Define the packages needed by the model and scripts
conda_dep = CondaDependencies()

# Installs numpy version 1.17.0 conda package
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")

# Installs pillow package
conda_dep.add_pip_package("azureml-sdk")
conda_dep.add_pip_package("xgboost")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep
run_config.environment = myenv

with open("myenv.yml","w") as f:
    f.write(conda_dep.serialize_to_string())

TODO: In the cell below, send a request to the web service you deployed to test it.

### Deploy the model as a Web Service on Azure Container Instance

In [26]:
# Use environment in InferenceConfig
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "liver", 'type': "classification"}, 
                                               description = 'Liver disease dataset for predicting whether a patient is suffering from liver disease or not')

aci_service_name = "mservice"
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
aci_service.update(enable_app_insights=True)
print(aci_service.state)

print(aci_service.scoring_uri)

print(aci_service.swagger_uri)

mservice
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://40ce93b6-2e99-407f-a64a-823e568f9665.southcentralus.azurecontainer.io/score
http://40ce93b6-2e99-407f-a64a-823e568f9665.southcentralus.azurecontainer.io/swagger.json


## Test
Now that the model is trained, run the test data through the trained model to get the predicted values. This calls the ACI web service to do the prediction.

Note that the JSON passed to the ACI web service is an array of rows of data. Each row should either be an array of values in the same order that was used for training or a dictionary where the keys are the same as the column names used for training. The example below uses dictionary rows.

In [35]:
#load the test dataset from the url
data_path = "https://raw.githubusercontent.com/chollette/Liver-Disease-Classification-Azure-ML-Capstone-Project/master/starter_file/data/test.csv"
df = pd.read_csv(data_path)

def clean_data(data):
    # Clean and one hot encode data
    
    #remove ?
    data.columns = data.columns.str.replace(r'?', '')
    #shorten column names    
    data.columns = ['Age', 'Gender', 'T_Bili', 'D_Bili', 'AA_Phosphate', 'SgptA_Aminotrans', 'SgotA_Aminotrans','T_proteins', 'ALB_Albumin', 'AG_AlbuminR_GlobulinR']
    # one hot encoding using pandas method (get_dummies)
    Gen = pd.get_dummies(data.Gender, prefix='Gender')
    data.drop("Gender", inplace=True, axis=1)
    data = data.join(Gen)
    data = data[['Age', 'Gender_Female', 'Gender_Male', 'T_Bili', 'D_Bili', 'AA_Phosphate', 'SgptA_Aminotrans', 'SgotA_Aminotrans','T_proteins', 'ALB_Albumin', 'AG_AlbuminR_GlobulinR']]
    return data  

x_test = clean_data(df)

#Sampling few rows (2) of the dataframe for testing
test_data = x_test[:2]
# Set the content type
#test_data = json.dumps({"data": test_data.values.tolist()})
test_data = test_data.to_json()
data = "{\"data\": " + test_data +"}"

#Predict using the deployed model
headers = {'Content-Type': 'application/json'}
resp = requests.post(aci_service.scoring_uri, data, headers=headers)

#print("POST to url", aci_service.scoring_uri)
print("prediction:", resp.text)

prediction: [2, 1]


## Get Logs from a Deployed Web Service
Gets logs from a deployed web service. The get_logs() function etrieves logs from a deployed web service. If there are errors  during deployment, the logs can catch such errors.

In [ ]:
TODO: In the cell below, print the logs of the web service and delete the service. 

In [37]:
print(aci_service.get_logs())

2021-01-29T11:47:36,204286500+00:00 - iot-server/run 
2021-01-29T11:47:36,220843600+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a3d94706eb0464a2ea83f306c60118d6/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a3d94706eb0464a2ea83f306c60118d6/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a3d94706eb0464a2ea83f306c60118d6/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a3d94706eb0464a2ea83f306c60118d6/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a3d94706eb0464a2ea83f306c60118d6/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-29T11:47:36,219964800+00:00 - gunicorn/run 
2021-01-29T11:47:36,238828700+00:00 - rsyslog/run 
EdgeHubC

## Delete Resources

### Delete a Web Service
Deletes the specified web service.

In [ ]:
aci_service.delete()

### Delete Compute Cluster
Deletes the specified compute target.

In [ ]:
compute_target.delete()